In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision
import numpy as np


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5,padding=2),#3代表输入通道，16代表输出通道（16个卷积核）
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, padding=2),  # 3代表输入通道，16代表输出通道（16个卷积核）
            nn.BatchNorm2d(128),  
            nn.ReLU(),
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=5, padding=2),  # 3代表输入通道，16代表输出通道（16个卷积核）
            nn.BatchNorm2d(256),  
            nn.ReLU(),
        )
        self.layer4 =nn.Sequential(
            nn.Conv2d(256, 64, kernel_size=3, padding=1),  # 3代表输入通道，16代表输出通道（16个卷积核）
            nn.BatchNorm2d(64),  
            nn.ReLU()
        )
        self.fc1 = nn.Linear(64 * 8 * 8, 100)
        #池化层
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        out = self.layer1(x)
        out = self.pool(self.layer2(out))
        out = self.pool(self.layer3(out))
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        return out

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def load_data():
    # 定义数据增强
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # 加载数据集
    trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=8,pin_memory=True)

    testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=8,pin_memory=True)

    # 加载验证集
    validset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
    validloader = torch.utils.data.DataLoader(validset, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)

    # 定义类别名称
    classes = tuple(trainset.classes)
    return trainloader,testloader,validloader,classes



def get_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net = CNN().to(device)
    return net,device

def get_config(net):
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
    return criterion,optimizer

def train_model(net, criterion, optimizer, trainloader, valloader, device,epochs=30):
    # Train the model
    net.to(device)
    train_loss_list = []
    train_acc_list = []
    val_loss_list = []
    val_acc_list = []
    train_loss_iter_list = []
    train_acc_iter_list = []
    for epoch in range(epochs):  # loop over the dataset multiple times
        print("Epoch[{}/{}]:".format(epoch + 1, epochs))
        running_train_loss = 0.0
        running_train_acc = 0.0
        running_train_iter_loss = 0.0
        running_train_iter_acc = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            acc = accuracy(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_train_iter_loss += loss.item()
            running_train_iter_acc += acc.item()
            running_train_loss += loss.item()
            running_train_acc += acc.item()

            if i % 100 == 99:  # print every 100 mini-batches
                print('[%d, %5d] train_loss: %.3f, train_accuracy: %.3f' %
                      (epoch + 1, i + 1, running_train_iter_loss / 100, running_train_iter_acc / 100))
                train_loss_iter_list.append(running_train_iter_loss / 100)
                train_acc_iter_list.append(running_train_iter_acc / 100)
                running_train_iter_loss = 0.0
                running_train_iter_acc = 0.0

        train_loss_list.append(running_train_loss / len(trainloader))
        train_acc_list.append(running_train_acc / len(trainloader))
        print('Epoch[ %d / %d ] : train_loss: %.3f, train_accuracy: %.3f' % (epoch + 1, epochs, running_train_loss / len(trainloader), running_train_acc / len(trainloader)))


        # Evaluate the model on the validation set
        running_val_loss = 0.0
        running_val_acc = 0.0
        with torch.no_grad():
            for data in valloader:
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = net(inputs)
                val_loss = criterion(outputs, labels)
                val_acc = accuracy(outputs, labels)
                running_val_loss += val_loss.item()
                running_val_acc += val_acc.item()
        val_loss_list.append(running_val_loss / len(valloader))
        val_acc_list.append(running_val_acc / len(valloader))
        print('Epoch[ %d / %d ] : val_loss: %.3f, val_accuracy: %.3f' %
              (epoch+1, epochs, running_val_loss / len(valloader), running_val_acc / len(valloader)))
        # 保存模型权重
        best_val_acc = max(val_acc_list)
        best_model_path = 'best_model.pth'
        if running_val_acc / len(valloader) == best_val_acc:
            torch.save(net.state_dict(), best_model_path)
            print(f'Saved best model to {best_model_path}')
    print('Finished Training')


    plt.plot(train_loss_list, label='train')
    plt.plot(val_loss_list, label='val')
    plt.title('Loss Curve')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    plt.savefig('loss.png')

    plt.plot(train_acc_list, label='train')
    plt.plot(val_acc_list, label='val')
    plt.title('Accuracy Curve')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    plt.savefig('accuracy.png')


def test_model(net,testloader,criterion,device):
    # Test the model
    correct = 0
    total = 0
    test_loss = 0.0
    predictions=[]
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            test_loss += criterion(outputs, labels).item()
            predictions.append(predicted.cpu().numpy())

    print('test accuracy: %d %%' % (100 * correct / total))
    print('Test loss: %.3f' % (test_loss / len(testloader)))

In [ ]:
trainloader, testloader,validloader,classes=load_data()
net,device = get_model()
print('current device:  ', device)
criterion,optimizer=get_config(net)
train_model(net, criterion, optimizer, trainloader, validloader, device)
test_model(net, testloader, criterion, device)